# Função Preditiva
 Notebook apenas com uma **função de predição**: recebe novos dados em forma de dicionário e retorna a predição de default. 

"""
Estrutura esperada para o dicionário de entrada (input_dict):
{
    'flag_valor_vencido': bin,  # 0 ou 1
    'quant_protestos': int,  # Número inteiro
    'default_3months': int,  # 0 ou 1
    'opcao_tributaria': str,  # Categoria (exemplo: 'Simples', 'Lucro Real')
    'razao_valor_vencido': float,  # Número decimal
    'forma_pagamento_agrup': str,  # Categoria
    'periodo_fiscal': str,  # Categoria
    'ioi_3months': int,  # Número inteiro
    'historico_pagamento': float  # Número decimal
}
"""

se no seu modelo final optou por usar as variáveis preditoras "ioi\_3months", "valor\_vencido" e "valor\_total\_pedido", espera-se que sua função de predição tenha como input um dicionário da forma
input_dict = {"ioi_3months": 3, "valor_vencido":125000, "valor_total_pedido":35000}
```
onde os valores no dicionário acima são os hipotéticos novos valores, e retorne um dicionário com o valor da predição:

```
{"default":0}
```
ou 

```
{"default":1}

### Bibliotecas

In [47]:
#adicionando retorno no diretório ao caminho
import sys
sys.path.append('../')

#ignorar warnings 
import warnings
warnings.filterwarnings('ignore')

#informação de diretórios
from x_health.config import *

In [48]:
import pickle
import pandas as pd
import json

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

## Leitura do Modelo Treinado

In [49]:
# Definir o caminho do modelo
caminho_modelo = f"{MODELS_DIR}/modelo_xgboost.pkl"

# Carregar o modelo treinado
with open(caminho_modelo, "rb") as file:
    modelo = pickle.load(file)

## Caminho do dicionário

In [51]:
# Definir o caminho do arquivo de entrada
caminho_arquivo_entrada = f"{RAW_DATA_DIR}\input_dados_random.json"

# Carregar os dados do arquivo JSON
with open(caminho_arquivo_entrada, "r") as file:
    input_dict = json.load(file)

In [52]:
# Converter dicionário para DataFrame
df = pd.DataFrame([input_dict])
df.columns

Index(['flag_valor_vencido', 'quant_protestos', 'default_3months',
       'opcao_tributaria', 'razao_valor_vencido', 'forma_pagamento_agrup',
       'periodo_fiscal', 'ioi_3months', 'historico_pagamento'],
      dtype='object')

In [53]:
# Converte variáveis categóricas para numéricas
label_encoders = {}
for col in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Salva os encoders para referência futura, se necessário

In [54]:
#garantir a ordem das colunas
df = df[modelo.feature_names]

In [55]:
# Converter DataFrame para DMatrix
dmatrix = xgb.DMatrix(df)

In [56]:
# Fazer predição
predicao = modelo.predict(dmatrix)

In [57]:
# Converter para 0 ou 1 baseado no threshold de 0.5
resultado = int(predicao[0] > 0.5)

In [58]:
# Retornar a predição
default_predicao = {"default": resultado}
print(default_predicao)

{'default': 0}


In [60]:
caminho_default_predicao = f"{PROCESSED_DATA_DIR}\default_predicao.json"
with open(caminho_arquivo_entrada, "w") as file:
    json.dump(default_predicao, file, indent=4)
    
    print(f"Arquivo salvo com sucesso em {caminho_default_predicao}!")

Arquivo salvo com sucesso em C:\Users\Natalia\OneDrive\Documentos\PROJETOS\Kognita\x_health\data\processed\default_predicao.json!
